In [1]:
from scipy.io import wavfile 
from scipy.signal import lfilter, convolve
import pyaudio                  # Reproducció d'àudio en Python
import IPython.display as ipd   # Widget per reproduir àudio en el Jupyter
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
%matplotlib widget

In [2]:
# Llegim el fitxer d'àudio
fitxer = "f1.wav"            # Canvieu-lo pel fitxer indicat a la pregunta 9
fm, s = wavfile.read(fitxer)
s = s / 2**15 #Normaliza y convierte a float64
print("Llegit fitxer amb freqüència de mostratge fm={} Hz".format(fm))
ipd.Audio(s, rate=fm, autoplay=False)                # Això permet reproduir l'àudio des del Jupyter. No és important: Ho podem obviar si no funciona.

Llegit fitxer amb freqüència de mostratge fm=8000 Hz


In [3]:
%matplotlib widget
plt.figure(1)
plt.title("Signal Wave...")
plt.plot(s)
plt.show()

FigureCanvasNbAgg()

In [4]:
%matplotlib widget
t99 = s[120*99:120*99+240]
t99 = t99 - np.mean(t99)
r99 = np.correlate(t99, t99, 'full')

plt.figure(2)
plt.plot(r99)
plt.show()

FigureCanvasNbAgg()

In [5]:
N = 239 - 183
print(N)
print(r99[183]/r99[239])
print(r99[239])

56
0.7796901523982371
0.42089052032291263


In [6]:
N = np.zeros(401)
sonora = np.zeros(401) # 0 sorda i 1 sonora
energies = np.zeros(401)

for i in range(401):
    ti = s[120*i:120*i+240]
    ti = ti - np.mean(ti)
    ri = np.correlate(ti, ti, 'full')
    
    rmin = np.argmin(ri[239:]) + 239
    rmax = np.argmax(ri[rmin:]) + rmin
    
    Ni = rmax - 239
    N[i] = Ni
    
    if ri[239+Ni]/ri[239] >= 0.5:
        sonora[i] = 1
    else:
        N[i] = 0
    
    energies[i] = ri[239]

#print(N)
#print(sonora)
#print(energies)

In [7]:
x = np.zeros(120)
x[0] = 1

coef = np.load('coeficientes.npz')
a = coef['a'] #coeficientes ai (i=0,...,P) con P = 10 y a0 = 1
b = coef['b']

h = lfilter([b[99]], a[99], x, axis=- 1, zi=None)

plt.figure(3)
plt.plot(h)
plt.show()

FigureCanvasNbAgg()

In [8]:
plt.figure(4)
plt.plot(t99)
plt.show()

FigureCanvasNbAgg()

In [9]:
y = convolve(t99, h)
plt.figure(5)
plt.plot(y)
plt.show()

FigureCanvasNbAgg()

In [10]:
x = np.zeros((401, 120))
i = 0
for j in range (401):
    if not sonora[j]:
        x[j] = np.random.normal(0, 1, 120)
    else:
        while(i < 120):
            if i >= 0:
                x[j, i] = sqrt(N[j])
            i += int(N[j])
        i = int(N[j+1]) - (120 - i + int(N[j]))

In [11]:
plt.figure(7)
plt.plot(x[99])
plt.show()

h99 = lfilter(b, a[99], x[99], axis=- 1, zi=None)
plt.figure(8)
plt.plot(h99)
plt.show()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [12]:
ss = []
for i in range (401):
    h1 = lfilter([b[i]], a[i], x[i], axis=- 1, zi=None)
    ss.extend(h1)
    
plt.figure(9)
plt.plot(ss)
plt.show()

ipd.Audio(ss, rate=fm, autoplay=False) 

FigureCanvasNbAgg()

In [13]:
ss2 = []
zf = np.zeros(10)
for i in range (401):
    h1, zf = lfilter([b[i]], a[i], x[i], axis=- 1, zi=zf)
    ss2.extend(h1)
    
plt.figure(10)
plt.plot(ss2)
plt.show()

ipd.Audio(ss2, rate=fm, autoplay=False) 

FigureCanvasNbAgg()

In [14]:
n = 120

ss2 = []
zf = np.zeros(10)
x = np.zeros((401, n))
i = 0
for j in range (401):
    if not sonora[j]:
        x[j] = np.random.normal(0, 1, n)
    else:
        while(i < n):
            if i >= 0:
                x[j, i] = sqrt(N[j])
            i += int(N[j])
        i = int(N[j+1]) - (120 - i + int(N[j]))
        
for i in range (401):
    h1, zf = lfilter([b[i]], a[i], x[i], axis=- 1, zi=zf)
    ss2.extend(h1)
    
plt.figure(11)
plt.plot(ss2)
plt.show()

ipd.Audio(ss2, rate=fm, autoplay=False) 

FigureCanvasNbAgg()